In [1]:
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

%matplotlib inline

In [2]:
mpl.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [4]:
listings_df = pd.read_csv('./listings.csv')
lat_lon_df = listings_df[['id', 'latitude', 'longitude', 'price']].copy()
lat_lon_df.head()

,id,latitude,longitude,price
0,12147973,42.282619,-71.133068,$250.00
1,3075044,42.286241,-71.134374,$65.00
2,6976,42.292438,-71.135765,$65.00
3,1436513,42.281106,-71.121021,$75.00
4,7651065,42.284512,-71.136258,$79.00
